In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
job_title = 'Business Analyst'

locations = ['Nairobi', 'Bomet']

first_site = 'https://www.myjobmag.co.ke/search/jobs/'
parameters = {
    'q': job_title,
    'location': '--'.join(locations)
}
link_template = 'https://www.myjobmag.co.ke/'

currentPage = 1
records = []


def add_records(tags, records):
    added_job = False
    for elem in tags:
        attrs = elem.attrs
        if 'href' not in attrs:
            continue
        href = attrs['href']
        if '/job/' in href:
            records.append({
                'title': elem.text,
                'link': f'{link_template}{href}'
            })
            added_job = True
    return added_job


while True:
    print(f'Current page: {currentPage}', end='\r')
    page = requests.get(
        url=first_site,
        params=parameters
    )

    soup = BeautifulSoup(page.content, 'html.parser')
    added_job = add_records(soup.find_all('a', recursive=True), records)
    if not added_job:
        break
    currentPage += 1
    parameters['currentPage'] = currentPage
jobs_table = pd.DataFrame.from_records(records)
jobs_table.to_csv('myjobmag_jobs.csv')
